# construct subset of objects

In [1]:
import os
import json

json_path = os.path.join('../../datasets', 'gqa', 'GQA_100', 'fewshot')


In [4]:
# load json

file_path = os.path.join(json_path, 'non_novel', 'non_novel_fewshot.json')
# objects = ('shirt', 'building', 'sign', 'car', 'flower', 'pants', 'helmet', 'pole', 'leaves', 'wall', 'door', 'fence', 'hat', 'shorts')      # fresh
objects = (('plate', 'grass', 'table', 'chair', 'jacket', 'shoe', 'bench'))      # old
new_name = 'nons.json'

info = json.load(open(file_path))


In [5]:

subset = []
for item in info:
    if len(set(item['comb']).difference(objects)) == 0:     # all objects in our desired set
        subset.append(item)

print(len(subset))
print(sorted(list(set([tuple(sorted(item['comb'])) for item in subset]))))

1000
[('bench', 'chair'), ('bench', 'jacket'), ('bench', 'table'), ('chair', 'grass'), ('chair', 'shoe'), ('grass', 'jacket'), ('grass', 'plate'), ('grass', 'shoe'), ('grass', 'table'), ('jacket', 'shoe')]


In [12]:
# save subset

# with open(os.path.join(json_path, new_name), 'w') as f:
#     json.dump(subset, f)

In [7]:
# for sys

file_path = os.path.join(json_path, 'sys', 'sys_fewshot.json')
objects = ('shirt', 'building', 'sign', 'car', 'flower', 'pants', 'helmet', 'pole', 'leaves', 'wall', 'door', 'fence', 'hat', 'shorts')      # fresh
# objects = (('plate', 'grass', 'table', 'chair', 'jacket', 'shoe', 'bench'))      # old
new_name = 'syss.json'

info = json.load(open(file_path))

subset = []
for item in info:
    if len(set(item['comb']).difference(objects)) == 0:     # all objects in our desired set
        subset.append(item)

print(len(subset))
print(sorted(list(set([tuple(sorted(item['comb'])) for item in subset]))))

# # save subset
#
# with open(os.path.join(json_path, new_name), 'w') as f:
#     json.dump(subset, f)

4200
[('building', 'car'), ('building', 'door'), ('building', 'fence'), ('building', 'helmet'), ('building', 'pants'), ('building', 'shirt'), ('building', 'shorts'), ('building', 'wall'), ('car', 'door'), ('car', 'fence'), ('car', 'hat'), ('car', 'pole'), ('car', 'wall'), ('door', 'flower'), ('door', 'hat'), ('door', 'helmet'), ('door', 'pants'), ('door', 'shirt'), ('door', 'shorts'), ('door', 'sign'), ('door', 'wall'), ('fence', 'hat'), ('fence', 'shirt'), ('fence', 'sign'), ('flower', 'hat'), ('flower', 'leaves'), ('flower', 'shorts'), ('flower', 'sign'), ('hat', 'helmet'), ('hat', 'leaves'), ('hat', 'pants'), ('helmet', 'pants'), ('leaves', 'pants'), ('leaves', 'pole'), ('pants', 'pole'), ('pants', 'sign'), ('pole', 'shorts'), ('pole', 'sign'), ('shirt', 'shorts'), ('shirt', 'sign'), ('shorts', 'wall'), ('sign', 'wall')]


# Generate results table

In [1]:
import numpy as np
import os
import pandas as pd

def get_accs(exp_name, mode='sys', project_name='CGQA', name='Top1_Acc_Stream/eval_phase/test_stream'):
    if not os.path.exists(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}.npy')):
        return np.array([[np.nan]])
    if mode == 'continual':
        results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}.npy'), allow_pickle=True)
    else:
        results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}-{mode}-naive-frz.npy'), allow_pickle=True)

    accs = []

    for k, v in results[-1].items():
        if k.startswith(name):
            accs.append(v)

    return np.array(accs)

# '''Load one exp'''
# exp_name = 'Naive-tsk'
# results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}.npy'), allow_pickle=True)
# last_result = results[-1]
# # print('Average test acc:', get_average_metric(results[-1], 'Top1_Acc_Stream/eval_phase/test_stream'))

In [15]:


modes = ['continual', 'sys', 'pro', 'sub', 'non', 'noc']
'''resnet18'''
# exps = ['naive-cls-lr0_003', 'er-cls-lr0_003', 'gem-cls-lr0_01-p32-m0_3', 'lwf-cls-lr0_005-a1-t1', 'ewc-cls-lr0_005-lambda0_1']
# exps_dis = ['Finetune', 'ER', 'GEM', 'LwF', 'EWC']
# exps = ['naive-tsk-lr0_008', 'er-tsk-lr0_0008', 'gem-tsk-lr0_001-p32-m0_3', 'lwf-tsk-lr0_01-a1-t1', 'ewc-tsk-lr0_005-lambda2']
# exps_dis = ['Finetune*', 'ER*', 'GEM*', 'LwF*', 'EWC*']

'''vit'''
# exps = ['ht-vit-naive-cls-lr0_0001', 'ht-vit-er-cls-lr0_0001', 'ht-vit-gem-cls-lr5e-05', 'ht-vit-lwf-cls-lr0_0001', 'ht-vit-ewc-cls-lr0_0001']
# exps_dis = ['Finetune', 'ER', 'GEM', 'LwF', 'EWC']
# exps = ['ht-vit-naive-tsk-lr0_0001', 'ht-vit-er-tsk-lr0_0001', 'ht-vit-gem-tsk-lr1e-05', 'ht-vit-lwf-tsk-lr0_0001', 'ht-vit-ewc-tsk-lr0_0001']
# exps_dis = ['Finetune*', 'ER*', 'GEM*', 'LwF*', 'EWC*']

'''concept'''
exps = [

    ]
exps_dis = ['Finetune', 'ER', 'GEM', 'LwF', 'EWC', 'Finetune*', 'ER*', 'GEM*', 'LwF*', 'EWC*']


collected_data = []
for exp_idx, exp_name in enumerate(exps):
    for mode in modes:
        accs = get_accs(exp_name, mode, 'CGQA')

        collected_data.append(pd.DataFrame({'Method': exps_dis[exp_idx], 'Phase': mode, 'Accuracy': accs}))

collected_data = pd.concat(collected_data, ignore_index=True)

In [13]:
'''cal mean and ci95, then formulate table str-cell and to latex.'''
# exp = 'Finetune'
# mode = 'continual'
# acc_list = collected_data[(collected_data['Method'] == exp) & (collected_data['Phase'] == mode)]['Accuracy']
data = []
for exp in exps_dis:
    for mode in modes:
        acc_list = collected_data[(collected_data['Method'] == exp) & (collected_data['Phase'] == mode)]['Accuracy']
        mean = acc_list.mean()
        ci95 = 1.96 * (acc_list.std()/np.sqrt(len(acc_list)))
        acc_str = f'{mean*100:.2f} +- {ci95*100:.2f}' if mode != 'continual' else f'{mean*100:.2f}'
        data.append(pd.DataFrame({'Method': exp, 'Phase': mode, 'mean': mean, 'ci95': ci95, 'str': acc_str}, index=[0]))
data = pd.concat(data, ignore_index=True)
# print(data)

# to latex

data = data.pivot(index='Method', values='str', columns='Phase')
data = data[modes]
data = data.reindex(exps_dis)
print(data.style.to_latex().replace('+-', '$\\pm$'))
data.head()

\begin{tabular}{lllllll}
Phase & continual & sys & pro & sub & non & noc \\
Method &  &  &  &  &  &  \\
Finetune & 8.59 & 50.49 $\pm$ 0.73 & 55.14 $\pm$ 0.82 & 60.91 $\pm$ 0.77 & 54.26 $\pm$ 0.73 & 48.44 $\pm$ 0.83 \\
ER & 10.58 & 47.63 $\pm$ 0.72 & 52.92 $\pm$ 0.84 & 59.47 $\pm$ 0.76 & 51.12 $\pm$ 0.79 & 47.23 $\pm$ 0.82 \\
GEM & 8.62 & 31.59 $\pm$ 0.75 & 34.21 $\pm$ 0.90 & 43.19 $\pm$ 0.90 & 34.41 $\pm$ 0.83 & 31.94 $\pm$ 0.77 \\
LwF & 10.58 & 54.26 $\pm$ 0.79 & 58.43 $\pm$ 0.77 & 64.23 $\pm$ 0.70 & 58.79 $\pm$ 0.75 & 51.26 $\pm$ 0.77 \\
EWC & 8.77 & 50.13 $\pm$ 0.79 & 53.56 $\pm$ 0.79 & 60.18 $\pm$ 0.75 & 53.63 $\pm$ 0.77 & 47.05 $\pm$ 0.79 \\
\end{tabular}



Phase,continual,sys,pro,sub,non,noc
Method,,,,,,
Finetune,8.59,50.49 +- 0.73,55.14 +- 0.82,60.91 +- 0.77,54.26 +- 0.73,48.44 +- 0.83
ER,10.58,47.63 +- 0.72,52.92 +- 0.84,59.47 +- 0.76,51.12 +- 0.79,47.23 +- 0.82
GEM,8.62,31.59 +- 0.75,34.21 +- 0.90,43.19 +- 0.90,34.41 +- 0.83,31.94 +- 0.77
LwF,10.58,54.26 +- 0.79,58.43 +- 0.77,64.23 +- 0.70,58.79 +- 0.75,51.26 +- 0.77
EWC,8.77,50.13 +- 0.79,53.56 +- 0.79,60.18 +- 0.75,53.63 +- 0.77,47.05 +- 0.79


In [16]:
'''cal mean and ci95 and relative score, then formulate table str-cell and to latex.'''
data = []
for exp in exps_dis:
    mean_dict = {}
    for mode in modes:
        acc_list = collected_data[(collected_data['Method'] == exp) & (collected_data['Phase'] == mode)]['Accuracy']
        mean = acc_list.mean()
        mean_dict[mode] = mean
        ci95 = 1.96 * (acc_list.std()/np.sqrt(len(acc_list)))
        acc_str = f'{mean*100:.2f} +- {ci95*100:.2f}' if mode != 'continual' else f'{mean*100:.2f}'
        data.append(pd.DataFrame({'Method': exp, 'Phase': mode, 'mean': mean, 'ci95': ci95, 'str': acc_str}, index=[0]))

    for mode, mean in mean_dict.items():
        score = (mean - mean_dict['non']) / mean_dict['non']
        data.append(pd.DataFrame({'Method': exp, 'Phase': f'{mode}_score', 'str': f'{score*100:+.2f}'}, index=[0]))

data = pd.concat(data, ignore_index=True)
# print(data)

# to latex

data = data.pivot(index='Method', values='str', columns='Phase')
data = data[['continual', 'continual_score', 'sys', 'sys_score', 'pro', 'pro_score', 'sub', 'sub_score', 'non', 'non_score',  'noc', 'noc_score']]
data = data.reindex(exps_dis)
print(data.style.to_latex().replace('+-', '$\\pm$'))
data.head()

\begin{tabular}{lllllllllllll}
Phase & continual & continual_score & sys & sys_score & pro & pro_score & sub & sub_score & non & non_score & noc & noc_score \\
Method &  &  &  &  &  &  &  &  &  &  &  &  \\
Finetune* & 44.84 & -17.22 & 50.61 $\pm$ 0.78 & -6.57 & 54.66 $\pm$ 0.78 & +0.91 & 60.45 $\pm$ 0.78 & +11.61 & 54.17 $\pm$ 0.77 & +0.00 & 47.69 $\pm$ 0.82 & -11.96 \\
ER* & 65.43 & +19.97 & 50.58 $\pm$ 0.74 & -7.26 & 55.90 $\pm$ 0.82 & +2.50 & 61.65 $\pm$ 0.75 & +13.04 & 54.54 $\pm$ 0.77 & +0.00 & 48.98 $\pm$ 0.82 & -10.19 \\
GEM* & 29.85 & +94.93 & 14.09 $\pm$ 0.47 & -7.99 & 16.67 $\pm$ 0.48 & +8.86 & 15.32 $\pm$ 0.53 & +0.02 & 15.31 $\pm$ 0.49 & +0.00 & 16.42 $\pm$ 0.52 & +7.25 \\
LwF* & 45.65 & -16.74 & 50.77 $\pm$ 0.80 & -7.40 & 54.90 $\pm$ 0.79 & +0.12 & 60.91 $\pm$ 0.73 & +11.08 & 54.83 $\pm$ 0.76 & +0.00 & 47.63 $\pm$ 0.80 & -13.13 \\
EWC* & 44.10 & -8.59 & 45.57 $\pm$ 0.78 & -5.54 & 47.70 $\pm$ 0.78 & -1.13 & 55.70 $\pm$ 0.80 & +15.44 & 48.25 $\pm$ 0.80 & +0.00 & 42.81 $\pm$ 

Phase,continual,continual_score,sys,sys_score,pro,pro_score,sub,sub_score,non,non_score,noc,noc_score
Method,,,,,,,,,,,,
Finetune*,44.84,-17.22,50.61 +- 0.78,-6.57,54.66 +- 0.78,+0.91,60.45 +- 0.78,+11.61,54.17 +- 0.77,+0.00,47.69 +- 0.82,-11.96
ER*,65.43,+19.97,50.58 +- 0.74,-7.26,55.90 +- 0.82,+2.50,61.65 +- 0.75,+13.04,54.54 +- 0.77,+0.00,48.98 +- 0.82,-10.19
GEM*,29.85,+94.93,14.09 +- 0.47,-7.99,16.67 +- 0.48,+8.86,15.32 +- 0.53,+0.02,15.31 +- 0.49,+0.00,16.42 +- 0.52,+7.25
LwF*,45.65,-16.74,50.77 +- 0.80,-7.40,54.90 +- 0.79,+0.12,60.91 +- 0.73,+11.08,54.83 +- 0.76,+0.00,47.63 +- 0.80,-13.13
EWC*,44.10,-8.59,45.57 +- 0.78,-5.54,47.70 +- 0.78,-1.13,55.70 +- 0.80,+15.44,48.25 +- 0.80,+0.00,42.81 +- 0.79,-11.26


# Concept in or not in the last train task

In [1]:
%cd ..

\\172.18.36.77\datasets\continual-learning-baselines


In [2]:
from datasets.cgqa import continual_training_benchmark
benchmark = continual_training_benchmark(
    n_experiences=10, return_task_id=True,
    seed=1234, shuffle=True,
    dataset_root='../datasets',
    memory_size=0,
)
class_exp = benchmark.original_classes_in_exp
maps = benchmark.label_info[2]
str_class_exp = []
for classes in class_exp:
    str_class_exp.append([maps[c] for c in classes])
print(str_class_exp)

[[('fence', 'flower'), ('door', 'grass'), ('leaves', 'shirt'), ('grass', 'table'), ('shoe', 'shorts'), ('hat', 'table'), ('leaves', 'wall'), ('chair', 'grass'), ('door', 'shoe'), ('fence', 'helmet')], [('chair', 'sign'), ('grass', 'shorts'), ('hat', 'plate'), ('pole', 'shirt'), ('grass', 'pants'), ('pants', 'shoe'), ('pole', 'wall'), ('bench', 'chair'), ('helmet', 'plate'), ('leaves', 'shoe')], [('bench', 'shorts'), ('flower', 'pole'), ('chair', 'helmet'), ('pants', 'shorts'), ('helmet', 'shorts'), ('helmet', 'shoe'), ('hat', 'jacket'), ('hat', 'shorts'), ('jacket', 'shoe'), ('fence', 'wall')], [('bench', 'helmet'), ('hat', 'shirt'), ('bench', 'sign'), ('plate', 'wall'), ('grass', 'plate'), ('helmet', 'pole'), ('door', 'leaves'), ('bench', 'pants'), ('grass', 'jacket'), ('jacket', 'pole')], [('car', 'jacket'), ('building', 'plate'), ('helmet', 'leaves'), ('pants', 'shirt'), ('car', 'leaves'), ('bench', 'leaves'), ('fence', 'pants'), ('bench', 'shirt'), ('fence', 'grass'), ('building', 

In [3]:
fresh_concepts = sorted(set([concept for concept_tuple in str_class_exp[-1] for concept in concept_tuple]))
old_concepts = sorted(set([concept for concept_tuple in benchmark.label_info[0] for concept in concept_tuple]) - set(fresh_concepts))
print(f'fresh_concepts: {fresh_concepts}.')
print(f'old_concepts: {old_concepts}.')

fresh_concepts: ['building', 'car', 'door', 'fence', 'flower', 'hat', 'helmet', 'leaves', 'pants', 'pole', 'shirt', 'shorts', 'sign', 'wall'].
old_concepts: ['bench', 'chair', 'grass', 'jacket', 'plate', 'shoe', 'table'].


In [4]:
from datasets.cgqa import fewshot_testing_benchmark

benchmark_non = fewshot_testing_benchmark(
    n_experiences=300, mode='non', n_way=10, n_shot=10, n_val=5, n_query=10, task_offset=1, seed=1234, dataset_root='../datasets')
# non(f)
nonf = []
for concept_tuple in benchmark_non.label_info[0]:
    is_target = True
    for concept in concept_tuple:
        if concept not in fresh_concepts:
            is_target = False
    if is_target:
        nonf.append(concept_tuple)
nonf = sorted(nonf)
print(f'nonf: {nonf}')
# non(o)
nono = []
for concept_tuple in benchmark_non.label_info[0]:
    is_target = True
    for concept in concept_tuple:
        if concept not in old_concepts:
            is_target = False
    if is_target:
        nono.append(concept_tuple)
nono = sorted(nono)
print(f'nono: {nono}')

benchmark_non = fewshot_testing_benchmark(
    n_experiences=300, mode='sys', n_way=10, n_shot=10, n_val=5, n_query=10, task_offset=1, seed=1234, dataset_root='../datasets')
# sys(f)
sysf = []
for concept_tuple in benchmark_non.label_info[0]:
    is_target = True
    for concept in concept_tuple:
        if concept not in fresh_concepts:
            is_target = False
    if is_target:
        sysf.append(concept_tuple)
sysf = sorted(sysf)
print(f'sysf: {sysf}')
# sys(o)
syso = []
for concept_tuple in benchmark_non.label_info[0]:
    is_target = True
    for concept in concept_tuple:
        if concept not in old_concepts:
            is_target = False
    if is_target:
        syso.append(concept_tuple)
syso = sorted(syso)
print(f'syso: {syso}')

nonf: [('building', 'hat'), ('building', 'leaves'), ('car', 'flower'), ('car', 'helmet'), ('car', 'leaves'), ('car', 'pants'), ('car', 'shirt'), ('car', 'shorts'), ('car', 'sign'), ('door', 'fence'), ('door', 'leaves'), ('door', 'pole'), ('fence', 'flower'), ('fence', 'helmet'), ('fence', 'leaves'), ('fence', 'pants'), ('fence', 'pole'), ('fence', 'shorts'), ('fence', 'wall'), ('flower', 'helmet'), ('flower', 'pants'), ('flower', 'pole'), ('flower', 'shirt'), ('flower', 'wall'), ('hat', 'shirt'), ('hat', 'shorts'), ('hat', 'sign'), ('hat', 'wall'), ('helmet', 'leaves'), ('helmet', 'pole'), ('helmet', 'shirt'), ('helmet', 'shorts'), ('helmet', 'sign'), ('leaves', 'shirt'), ('leaves', 'shorts'), ('leaves', 'sign'), ('leaves', 'wall'), ('pants', 'shirt'), ('pants', 'shorts'), ('pants', 'wall'), ('pole', 'shirt'), ('pole', 'wall'), ('shirt', 'wall')]
nono: [('bench', 'chair'), ('bench', 'jacket'), ('bench', 'table'), ('chair', 'grass'), ('chair', 'shoe'), ('grass', 'jacket'), ('grass', 'pl

In [5]:
'''Generate fixed_class_order for 4 subset'''
import numpy as np
from datasets.cgqa import fewshot_testing_benchmark

# nonf
benchmark_non = fewshot_testing_benchmark(
    n_experiences=300, mode='non', n_way=10, n_shot=10, n_val=5, n_query=10, task_offset=1, seed=1234, dataset_root='../datasets')

maps = benchmark_non.label_info[1]      # from str to int
nonf_int = [maps[c] for c in nonf]
n_experiences = 300
selected_classes_in_exp = []
rng = np.random.RandomState(seed=1234)
for exp_idx in range(n_experiences):
    '''select n_way classes for each exp'''
    selected_class_idxs = list(rng.choice(nonf_int, 2, replace=False))   # 2-way
    selected_classes_in_exp.extend(selected_class_idxs)

print(f'nonf: {selected_classes_in_exp}')

# nono
nono_int = [maps[c] for c in nono]
n_experiences = 300
selected_classes_in_exp = []
rng = np.random.RandomState(seed=1234)
for exp_idx in range(n_experiences):
    '''select n_way classes for each exp'''
    selected_class_idxs = list(rng.choice(nono_int, 2, replace=False))   # 2-way
    selected_classes_in_exp.extend(selected_class_idxs)

print(f'nono: {selected_classes_in_exp}')

# sysf
benchmark_sys = fewshot_testing_benchmark(
    n_experiences=300, mode='sys', n_way=10, n_shot=10, n_val=5, n_query=10, task_offset=1, seed=1234, dataset_root='../datasets')

maps = benchmark_sys.label_info[1]      # from str to int
sysf_int = [maps[c] for c in sysf]
n_experiences = 300
selected_classes_in_exp = []
rng = np.random.RandomState(seed=1234)
for exp_idx in range(n_experiences):
    '''select n_way classes for each exp'''
    selected_class_idxs = list(rng.choice(sysf_int, 2, replace=False))   # 2-way
    selected_classes_in_exp.extend(selected_class_idxs)

print(f'sysf: {selected_classes_in_exp}')

# syso
syso_int = [maps[c] for c in syso]
n_experiences = 300
selected_classes_in_exp = []
rng = np.random.RandomState(seed=1234)
for exp_idx in range(n_experiences):
    '''select n_way classes for each exp'''
    selected_class_idxs = list(rng.choice(syso_int, 2, replace=False))   # 2-way
    selected_classes_in_exp.extend(selected_class_idxs)

print(f'syso: {selected_classes_in_exp}')

nonf: [26, 43, 67, 17, 49, 19, 76, 66, 70, 50, 81, 86, 22, 51, 84, 75, 66, 42, 25, 94, 26, 47, 75, 44, 15, 52, 76, 96, 67, 76, 36, 70, 83, 48, 15, 81, 17, 85, 76, 47, 85, 66, 96, 46, 36, 76, 73, 50, 26, 36, 51, 38, 46, 70, 48, 73, 84, 49, 88, 44, 86, 40, 43, 53, 84, 51, 15, 19, 51, 48, 89, 34, 66, 97, 83, 73, 19, 23, 76, 26, 53, 88, 19, 70, 34, 36, 64, 15, 66, 20, 64, 51, 89, 42, 81, 66, 15, 67, 96, 48, 51, 94, 89, 81, 25, 23, 94, 46, 97, 51, 64, 86, 53, 86, 88, 84, 69, 83, 44, 38, 17, 96, 96, 50, 46, 25, 34, 50, 49, 89, 69, 49, 85, 97, 23, 83, 20, 48, 75, 36, 81, 17, 66, 81, 53, 48, 26, 51, 43, 89, 50, 24, 73, 97, 49, 38, 94, 50, 22, 64, 40, 44, 36, 22, 19, 43, 38, 67, 86, 49, 48, 69, 36, 96, 46, 49, 15, 70, 40, 53, 72, 15, 69, 75, 49, 67, 25, 22, 66, 86, 52, 15, 38, 40, 94, 22, 36, 44, 97, 64, 15, 89, 66, 76, 17, 94, 36, 50, 52, 75, 88, 43, 72, 38, 38, 40, 83, 49, 67, 75, 85, 26, 46, 67, 20, 46, 48, 72, 83, 97, 34, 76, 20, 24, 23, 46, 53, 26, 96, 86, 20, 38, 83, 47, 20, 72, 86, 42, 8

# Generate results table for LMC

In [6]:
import numpy as np
import os
import pandas as pd

exp_name = 'lmc_resnet18_cgqa_dp5'
# exp_name = 'mntdp_resnet18_cgqa_dp5'

results = np.load(os.path.join('../../LMC-experiments', exp_name, 'results-continual.npy'))

results_sys = np.load(os.path.join('../../LMC-experiments', exp_name, 'results-sys.npy'))
results_pro = np.load(os.path.join('../../LMC-experiments', exp_name, 'results-pro.npy'))
results_sub = np.load(os.path.join('../../LMC-experiments', exp_name, 'results-sub.npy'))
results_non = np.load(os.path.join('../../LMC-experiments', exp_name, 'results-non.npy'))
results_noc = np.load(os.path.join('../../LMC-experiments', exp_name, 'results-noc.npy'))

print(f'continual: {results.mean()*100:.2f} & {(results.mean() - results_non.mean()) / results_non.mean():+.2f}')
print(f'sys: {results_sys.mean()*100:.2f} $\\pm$ {1.96 * (results_sys.std()/np.sqrt(len(results_sys)))*100:.2f} & {(results_sys.mean() - results_non.mean()) / results_non.mean():+.2f}')
print(f'pro: {results_pro.mean()*100:.2f} $\\pm$ {1.96 * (results_pro.std()/np.sqrt(len(results_pro)))*100:.2f} & {(results_pro.mean() - results_non.mean()) / results_non.mean():+.2f}')
print(f'sub: {results_sub.mean()*100:.2f} $\\pm$ {1.96 * (results_sub.std()/np.sqrt(len(results_sub)))*100:.2f} & {(results_sub.mean() - results_non.mean()) / results_non.mean():+.2f}')
print(f'non: {results_non.mean()*100:.2f} $\\pm$ {1.96 * (results_non.std()/np.sqrt(len(results_non)))*100:.2f} & {(results_non.mean() - results_non.mean()) / results_non.mean():+.2f}')
print(f'noc: {results_noc.mean()*100:.2f} $\\pm$ {1.96 * (results_noc.std()/np.sqrt(len(results_noc)))*100:.2f} & {(results_noc.mean() - results_non.mean()) / results_non.mean():+.2f}')


continual: 21.73 & -0.55
sys: 43.96 $\pm$ 0.89 & -0.09
pro: 49.38 $\pm$ 0.79 & +0.02
sub: 44.20 $\pm$ 0.77 & -0.08
non: 48.18 $\pm$ 0.81 & +0.00
noc: 28.96 $\pm$ 0.73 & -0.40


In [3]:
import torch

checkpoint = torch.load(os.path.join('../../LMC-experiments', 'mntdp_resnet18_cgqa_lr1e-3_', 'model.pt'), map_location=torch.device('cuda'))

checkpoint['structure_pool'] = [[0, 0, 0, 0], [1, 1, 1, 1], [1, 1, 2, 2], [1, 2, 3, 3], [1, 3, 4, 4], [0, 4, 5, 5], [1, 5, 6, 6], [1, 6, 7, 7], [2, 7, 8, 8], [1, 8, 9, 9]]

torch.save(checkpoint, os.path.join('../../LMC-experiments', 'mntdp_resnet18_cgqa_lr1e-3_', 'model_.pt'))

In [4]:
print(checkpoint['state_dict'].keys())

odict_keys(['_n_modules', 'min_temp_buffer', '_log_at_this_iter_flag_buffer', '_steps_since_last_addition', 'components.0.0.module_learned_buffer', 'components.0.0.bn_eval_buffer', 'components.0.0.module.0.conv1.weight', 'components.0.0.module.0.bn1.weight', 'components.0.0.module.0.bn1.bias', 'components.0.0.module.0.bn1.running_mean', 'components.0.0.module.0.bn1.running_var', 'components.0.0.module.0.bn1.num_batches_tracked', 'components.0.0.module.0.conv2.weight', 'components.0.0.module.0.bn2.weight', 'components.0.0.module.0.bn2.bias', 'components.0.0.module.0.bn2.running_mean', 'components.0.0.module.0.bn2.running_var', 'components.0.0.module.0.bn2.num_batches_tracked', 'components.0.0.module.1.conv1.weight', 'components.0.0.module.1.bn1.weight', 'components.0.0.module.1.bn1.bias', 'components.0.0.module.1.bn1.running_mean', 'components.0.0.module.1.bn1.running_var', 'components.0.0.module.1.bn1.num_batches_tracked', 'components.0.0.module.1.conv2.weight', 'components.0.0.module.

# Learn new dataset

In [3]:
import torch
from torch.utils.data.dataset import TensorDataset
from avalanche.benchmarks.utils import AvalancheDataset

# Create a dataset of 100 data points described by 22 features + 1 class label
x_data = torch.rand(100, 22)
y_data = torch.randint(0, 5, (100,))

# Create the Dataset
torch_data = TensorDataset(x_data, y_data)

avl_data = AvalancheDataset(torch_data)

sub_data = avl_data.subset(list(range(50)))
print(len(sub_data))  # 50
print(len(avl_data))  # 100, original data stays the same

from avalanche.benchmarks.utils import make_classification_dataset

# first, we add targets to the dataset. This will be used by the AvalancheDataset
# If possible, avalanche tries to extract the targets from the dataset.
# most datasets in torchvision already have a targets field so you don't need this step.
torch_data.targets = torch.randint(0, 5, (100,)).tolist()
tls = [0 for _ in range(100)] # one task label for each sample
sup_data = make_classification_dataset(torch_data, task_labels=tls)

print(sup_data.targets.name, len(sup_data.targets._data))
print(sup_data.targets_task_labels.name, len(sup_data.targets_task_labels._data))

# after subsampling
sub_data = sup_data.subset(range(10))
print(sub_data.targets.name, len(sub_data.targets._data))
print(sub_data.targets_task_labels.name, len(sub_data.targets_task_labels._data))

# after concat
cat_data = sup_data.concat(sup_data)
print(cat_data.targets.name, len(cat_data.targets._data))
print(cat_data.targets_task_labels.name, len(cat_data.targets_task_labels._data))

50
100
targets 100
targets_task_labels 100
targets 10
targets_task_labels 10
targets 200
targets_task_labels 200
